In [53]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')


In [54]:
from numba import jit  # pip install numba


In [55]:

@jit(nopython=True)
def run_pde_steps(C, nx, ny, nt, dx, dt, vx, vy, k, source_mask, D):
    """
    Numba-compiled PDE stepper. 
    Now explicitly accepts D to ensure Soft PINN and Hard PINN match physics.
    """
    for n in range(nt):
        C_old = C.copy()
        for i in range(1, ny-1):
            for j in range(1, nx-1):
                # Diffusion (Laplacian)
                d2x = (C_old[i, j+1] + C_old[i, j-1] - 2*C_old[i, j]) / dx**2
                d2y = (C_old[i+1, j] + C_old[i-1, j] - 2*C_old[i, j]) / dx**2
                
                # Advection (Upwind Gradient)
                # Simple central difference for smoothness, or upwind for stability
                grad_x = (C_old[i, j] - C_old[i, j-1]) / dx
                grad_y = (C_old[i, j] - C_old[i-1, j]) / dx
                
                # Discrete Update
                C[i, j] = C_old[i, j] + dt * (
                    D * (d2x + d2y) -        # Explicit Diffusion
                    vx * grad_x -            # Advection X
                    vy * grad_y -            # Advection Y
                    k * C_old[i, j] +        # Decay
                    source_mask[i, j]        # Source Input
                )
    return C

class PDESolver:
    def __init__(self, wind_speed, wind_angle, D=0.1, k=0.1):
        # NOTE: D=0.1 and v=1.0 gives Pe = vL/D = 10 (Matches Paper's Physics)
        self.wind_speed = wind_speed
        self.wind_angle = wind_angle
        self.D = D 
        self.k = k
        
        self.nx = 30
        self.ny = 30
        self.dx = 1.0 / (self.nx - 1)
        
        # Velocity Components
        self.vx = wind_speed * np.cos(wind_angle * np.pi / 180)
        self.vy = wind_speed * np.sin(wind_angle * np.pi / 180)
        v_max = np.sqrt(self.vx**2 + self.vy**2)
        
        # Stability Constraints (CFL)
        dt_diff = 0.2 * self.dx**2 / (self.D + 1e-8)
        dt_adv = 0.8 * self.dx / (v_max + 1e-8) if v_max > 0 else dt_diff
        self.dt = min(dt_diff, dt_adv)
        self.nt = int(0.5 / self.dt) # Simulate for 0.5 time units

    def solve(self, xs, ys):
        C = np.zeros((self.ny, self.nx))
        
        # Source Term (Gaussian Puff)
        x = np.linspace(0, 1, self.nx)
        y = np.linspace(0, 1, self.ny)
        X, Y = np.meshgrid(x, y)
        sigma = 2 * self.dx
        source = np.exp(-((X - xs)**2 + (Y - ys)**2) / (2*sigma**2))
        # Normalize source intensity
        source = source / (np.sum(source) * self.dx**2 + 1e-10)
        
        # Run Simulation
        C = run_pde_steps(C, self.nx, self.ny, self.nt, self.dx, self.dt, 
                          self.vx, self.vy, self.k, source, self.D)
        
        return np.maximum(C, 0)


In [ ]:
"""
# =============================================================================
# CORE COMPONENTS
# =============================================================================

class PDESolver:

    def __init__(self, wind_speed, wind_angle, D=1e-9, k=0.1):
        self.wind_speed = wind_speed
        self.wind_angle = wind_angle
        self.D = D
        self.k = k
        
        self.nx = 30
        self.ny = 30
        self.dx = 1.0 / (self.nx - 1)
        
        vx = wind_speed * np.cos(wind_angle * np.pi / 180)
        vy = wind_speed * np.sin(wind_angle * np.pi / 180)
        v_max = np.sqrt(vx**2 + vy**2)
        
        dt_diff = 0.2 * self.dx**2
        dt_adv = 0.8 * self.dx / (v_max + 1e-10) if v_max > 0 else dt_diff
        
        self.dt = min(dt_diff, dt_adv)
        self.nt = int(0.5 / self.dt)
        
        self.vx = vx
        self.vy = vy
        
    def solve(self, xs, ys):
       
        nx, ny, nt = self.nx, self.ny, self.nt
        dx, dt = self.dx, self.dt
        
        C = np.zeros((ny, nx))
        
        x = np.linspace(0, 1, nx)
        y = np.linspace(0, 1, ny)
        X, Y = np.meshgrid(x, y)
        
        sigma = 3 * dx
        source = np.exp(-((X - xs)**2 + (Y - ys)**2) / (2*sigma**2))
        source = source / (np.sum(source) * dx**2 + 1e-10)
        
        for n in range(nt):
            C_old = C.copy()
            
            C[1:-1, 1:-1] = (
                C_old[1:-1, 1:-1] +
                dt * (
                    (C_old[1:-1, 2:] + C_old[1:-1, :-2] - 2*C_old[1:-1, 1:-1]) / dx**2 +
                    (C_old[2:, 1:-1] + C_old[:-2, 1:-1] - 2*C_old[1:-1, 1:-1]) / dx**2 -
                    self.vx * (C_old[1:-1, 1:-1] - C_old[1:-1, :-2]) / dx -
                    self.vy * (C_old[1:-1, 1:-1] - C_old[:-2, 1:-1]) / dx -
                    self.k * C_old[1:-1, 1:-1] +
                    source[1:-1, 1:-1]
                )
            )
            
            C[0, :] = C[-1, :] = C[:, 0] = C[:, -1] = 0
            C = np.maximum(C, 0)
        
        return C
"""

In [56]:
class SensorNetwork:
    def __init__(self, n_sensors=9):
        self.n_sensors = n_sensors
        self.locations = []
        n_side = int(np.sqrt(n_sensors))
        for i in range(n_side):
            for j in range(n_side):
                self.locations.append([(i+1)/(n_side+1), (j+1)/(n_side+1)])
        self.locations = np.array(self.locations)
    
    def measure(self, C, noise=0.05):
        measurements = []
        ny, nx = C.shape
        for sx, sy in self.locations:
            i = int(sx * (nx - 1))
            j = int(sy * (ny - 1))
            val = C[j, i]
            if noise > 0:
                val += np.random.normal(0, noise * (np.max(C) + 1e-10))
            measurements.append(max(0, val))
        return np.array(measurements)



In [57]:
class MLP:
    def __init__(self, input_dim):
        self.W1 = np.random.randn(input_dim, 32) * 0.1
        self.b1 = np.zeros(32)
        self.W2 = np.random.randn(32, 16) * 0.1
        self.b2 = np.zeros(16)
        self.W3 = np.random.randn(16, 2) * 0.1
        self.b3 = np.zeros(2)
        self.trained = False
        self.train_losses = []
        self.val_losses = []
    
    def relu(self, x): return np.maximum(0, x)
    def sigmoid(self, x): return 1 / (1 + np.exp(-np.clip(x, -10, 10)))
    
    def forward(self, x):
        h1 = self.relu(x @ self.W1 + self.b1)
        h2 = self.relu(h1 @ self.W2 + self.b2)
        return self.sigmoid(h2 @ self.W3 + self.b3)

    def train(self, X_train, Y_train, X_val, Y_val, epochs=100, lr=0.01):
        self.X_mean = np.mean(X_train, axis=0)
        self.X_std = np.std(X_train, axis=0) + 1e-8
        self.Y_min = np.min(Y_train, axis=0)
        self.Y_max = np.max(Y_train, axis=0)
        
        X_train_norm = (X_train - self.X_mean) / self.X_std
        Y_train_norm = (Y_train - self.Y_min) / (self.Y_max - self.Y_min + 1e-8)
        X_val_norm = (X_val - self.X_mean) / self.X_std
        Y_val_norm = (Y_val - self.Y_min) / (self.Y_max - self.Y_min + 1e-8)
        
        batch_size = 32
        n_train = len(X_train)

        for epoch in range(epochs):
        # ADD THIS LINE - Learning rate decay
            current_lr = lr * (0.95 ** (epoch // 10))
            
            indices = np.random.permutation(n_train)
            X_shuffled = X_train_norm[indices]
            Y_shuffled = Y_train_norm[indices]
        
            epoch_loss = 0
            for i in range(0, n_train, batch_size):
                X_batch = X_shuffled[i:i+batch_size]
                Y_batch = Y_shuffled[i:i+batch_size]
        
        
                
                # Forward
                h1 = self.relu(X_batch @ self.W1 + self.b1)
                h2 = self.relu(h1 @ self.W2 + self.b2)
                out = self.sigmoid(h2 @ self.W3 + self.b3)
                
                # Backprop
                grad = (out - Y_batch) * out * (1 - out)
                
                dW3 = (h2.T @ grad) / len(X_batch)
                db3 = np.sum(grad, axis=0) / len(X_batch)
                grad2 = (grad @ self.W3.T) * (h2 > 0)
                dW2 = (h1.T @ grad2) / len(X_batch)
                db2 = np.sum(grad2, axis=0) / len(X_batch)
                grad1 = (grad2 @ self.W2.T) * (h1 > 0)
                dW1 = (X_batch.T @ grad1) / len(X_batch)
                db1 = np.sum(grad1, axis=0) / len(X_batch)
                
                #self.W3 -= lr * dW3; self.b3 -= lr * db3
                #self.W2 -= lr * dW2; self.b2 -= lr * db2
                #self.W1 -= lr * dW1; self.b1 -= lr * db1
                self.W3 -= current_lr * dW3; self.b3 -= current_lr * db3
                self.W2 -= current_lr * dW2; self.b2 -= current_lr * db2
                self.W1 -= current_lr * dW1; self.b1 -= current_lr * db1
                epoch_loss += np.mean((out - Y_batch)**2)
            
            self.train_losses.append(epoch_loss / (n_train/batch_size))
            val_out = self.forward(X_val_norm)
            self.val_losses.append(np.mean((val_out - Y_val_norm)**2))
            
            if (epoch+1) % 50 == 0:
                print(f"  Epoch {epoch+1}: Val Loss {self.val_losses[-1]:.5f}")
        self.trained = True

    def predict(self, X):
        if not self.trained: return np.full((len(X), 2), 0.5)
        X_norm = (X - self.X_mean) / self.X_std
        Y_norm = self.forward(X_norm)
        return Y_norm * (self.Y_max - self.Y_min) + self.Y_min

In [58]:
class PINN_hard:
    """Direct Inverse Solver (The 'Gold Standard')"""
    def __init__(self, solver, sensors):
        self.solver = solver
        self.sensors = sensors
    
    def loss(self, pos, measurements):
        xs, ys = pos
        if not (0.1 < xs < 0.9 and 0.1 < ys < 0.9): return 1e8
        C = self.solver.solve(xs, ys)
        pred = self.sensors.measure(C, noise=0)
        return np.sum((pred - measurements)**2)
    
    def predict(self, measurements):
        best, best_loss = None, np.inf
        # Try multiple starting points to avoid local minima
        for x0 in [[0.5, 0.5], [0.2, 0.2], [0.8, 0.8]]:
            res = minimize(lambda p: self.loss(p, measurements), x0, 
                           method='Nelder-Mead', options={'maxiter': 40})
            if res.fun < best_loss: best_loss = res.fun; best = res
        return best.x if best else np.array([0.5, 0.5])



In [59]:
class PINN:
    """Soft-Constrained PINN (The 'Paper Method')"""
    def __init__(self, solver, sensors, n_collocation=40):
        self.solver = solver
        self.sensors = sensors
        self.n_collocation = n_collocation
        #self.lambda_pde = 0.1
        self.lambda_pde = 5.0

                # Fixed collocation grid (deterministic)
        grid_size = int(np.sqrt(n_collocation))
        x_coll = np.linspace(0.1, 0.9, grid_size)
        y_coll = np.linspace(0.1, 0.9, grid_size)
        X_coll, Y_coll = np.meshgrid(x_coll, y_coll)
        
        # Convert to indices
        self.coll_i = (Y_coll * (solver.ny - 1)).astype(int).flatten()
        self.coll_j = (X_coll * (solver.nx - 1)).astype(int).flatten()
    
    def compute_pde_residual(self, C, dx):
        """Compute PDE residual with error handling"""
        ny, nx = C.shape
        
        # Ensure we have valid indices
        if nx < 3 or ny < 3:
            return 0.0  # Can't compute derivatives on tiny grid
        
        # Random collocation points (avoid boundaries)
        try:
            cols = np.random.randint(1, nx-1, self.n_collocation)
            rows = np.random.randint(1, ny-1, self.n_collocation)
        except ValueError:
            return 0.0  # Grid too small
        
        residuals = []
        for i, j in zip(rows, cols):
            # Check bounds (safety)
            if i <= 0 or i >= ny-1 or j <= 0 or j >= nx-1:
                continue
                
            # Compute derivatives
            laplacian = (C[i+1,j] + C[i-1,j] + C[i,j+1] + C[i,j-1] - 4*C[i,j]) / (dx**2)
            grad_x = (C[i,j+1] - C[i,j-1]) / (2*dx)
            grad_y = (C[i+1,j] - C[i-1,j]) / (2*dx)
            
            # PDE residual: ∂c/∂t = D∇²c - v·∇c - kc
            # At steady state, ∂c/∂t ≈ 0, so residual should be ≈ 0
            residual = (self.solver.D * laplacian) - \
                    (self.solver.vx * grad_x + self.solver.vy * grad_y) - \
                    (self.solver.k * C[i,j])
            
            residuals.append(residual**2)
        
        # Return mean squared residual (or 0 if no valid points)
        if len(residuals) == 0:
            return 0.0
        
        return np.mean(residuals)
    def loss(self, pos, measurements):
        xs, ys = pos
        
        # Boundary check
        if not (0.1 < xs < 0.9 and 0.1 < ys < 0.9): 
            return 1e8
        
        # Solve PDE
        try:
            C = self.solver.solve(xs, ys)
        except Exception:
            return 1e8  # If solver fails, return huge loss
        
        # Data loss
        pred = self.sensors.measure(C, noise=0)
        L_data = np.sum((pred - measurements)**2)
        
        # Physics loss
        L_pde = self.compute_pde_residual(C, self.solver.dx)
        
        # Check for None or NaN
        if L_pde is None or np.isnan(L_pde):
            L_pde = 0.0
        
        return L_data + self.lambda_pde * L_pde
    def predict(self, measurements):
        best, best_loss = None, np.inf
        for x0 in [[0.5, 0.5], [0.3, 0.7], [0.7, 0.3]]:
            res = minimize(lambda p: self.loss(p, measurements), x0, 
                           method='Nelder-Mead', options={'maxiter': 30})
            if res.fun < best_loss: best_loss = res.fun; best = res
        return best.x if best else np.array([0.5, 0.5])

In [60]:


class ParticleFilter:
    def __init__(self, solver, sensors):
        self.solver = solver
        self.sensors = sensors
        self.n_particles = 50
    
    def predict(self, measurements):
        particles = np.random.uniform(0.2, 0.8, (self.n_particles, 2))
        likelihoods = []
        for xs, ys in particles:
            C = self.solver.solve(xs, ys)
            error = np.sum((self.sensors.measure(C, noise=0) - measurements)**2)
            likelihoods.append(np.exp(-error / 0.05))
        weights = np.array(likelihoods) / (np.sum(likelihoods) + 1e-10)
        return np.average(particles, axis=0, weights=weights)

In [61]:

# =============================================================================
# LEARNABILITY ANALYSIS
# =============================================================================

def analyze_learnability(solver, sensors):
    """Check if problem is actually learnable"""
    
    print("\n" + "="*70)
    print(" LEARNABILITY ANALYSIS ".center(70))
    print("="*70)
    
    print("\n1. Sensor Discrimination Test")
    sources = [
        (0.3, 0.3), (0.3, 0.7), (0.7, 0.3), (0.7, 0.7),
        (0.5, 0.3), (0.5, 0.7), (0.3, 0.5), (0.7, 0.5),
        (0.5, 0.5)
    ]
    
    measurements_list = []
    for xs, ys in sources:
        C = solver.solve(xs, ys)
        meas = sensors.measure(C, noise=0)
        measurements_list.append(meas)
    
    n = len(sources)
    distances = []
    for i in range(n):
        for j in range(i+1, n):
            dist = np.linalg.norm(measurements_list[i] - measurements_list[j])
            distances.append(dist)
    
    min_dist = np.min(distances)
    max_dist = np.max(distances)
    mean_dist = np.mean(distances)
    
    print(f"  Min distance: {min_dist:.6f}")
    print(f"  Max distance: {max_dist:.6f}")
    print(f"  Mean distance: {mean_dist:.6f}")
    
    if min_dist / max_dist > 0.1:
        print("  ✓ Sources well-separated")
    else:
        print("  ⚠️  Some sources hard to distinguish")
    
    print("\n2. Noise Robustness Test")
    test_source = (0.4, 0.6)
    C_clean = solver.solve(*test_source)
    meas_clean = sensors.measure(C_clean, noise=0)
    
    for noise_level in [0.01, 0.05, 0.1]:
        noisy_samples = []
        for _ in range(10):
            meas_noisy = sensors.measure(C_clean, noise=noise_level)
            noisy_samples.append(meas_noisy)
        std_dev = np.std(noisy_samples, axis=0)
        snr = np.mean(meas_clean) / (np.mean(std_dev) + 1e-10)
        print(f"  Noise {noise_level*100:.0f}%: SNR = {snr:.2f}")
    
    print("\n3. Inverse Problem Uniqueness")
    grid_size = 5
    grid_sources = []
    grid_measurements = []
    
    for i in range(grid_size):
        for j in range(grid_size):
            xs = 0.2 + i * 0.6 / (grid_size - 1)
            ys = 0.2 + j * 0.6 / (grid_size - 1)
            C = solver.solve(xs, ys)
            meas = sensors.measure(C, noise=0)
            grid_sources.append([xs, ys])
            grid_measurements.append(meas)
    
    grid_sources = np.array(grid_sources)
    grid_measurements = np.array(grid_measurements)
    
    spatial_dist = np.linalg.norm(
        grid_sources[:, None, :] - grid_sources[None, :, :], axis=2
    )
    meas_dist = np.linalg.norm(
        grid_measurements[:, None, :] - grid_measurements[None, :, :], axis=2
    )
    
    mask = spatial_dist > 0
    correlation = np.corrcoef(spatial_dist[mask].flatten(), 
                             meas_dist[mask].flatten())[0, 1]
    
    print(f"  Correlation: {correlation:.3f}")
    if correlation > 0.7:
        print("  ✓ Problem well-posed")
    elif correlation > 0.4:
        print("  ~ Moderate - learnable but challenging")
    else:
        print("  ⚠️  Weak - may be ill-posed")
    
    return {
        'min_distance': min_dist,
        'max_distance': max_dist,
        'correlation': correlation
    }


### Figures

In [62]:

# =============================================================================
# INDIVIDUAL PLOTTING FUNCTIONS
# =============================================================================

def plot_fig1_training_curves(mlp):
    """Figure 1: Training convergence and overfitting check"""
    plt.figure(figsize=(8, 5))
    plt.plot(mlp.train_losses, label='Training Loss', alpha=0.8, linewidth=2, color='#2E86AB')
    plt.plot(mlp.val_losses, label='Validation Loss', alpha=0.8, linewidth=2, color='#A23B72')
    plt.xlabel('Epoch', fontsize=14, fontweight='bold')
    plt.ylabel('Loss (MSE)', fontsize=14, fontweight='bold')
    plt.title('MLP Training Convergence', fontsize=16, fontweight='bold', pad=15)
    plt.legend(fontsize=12, loc='upper right')
    plt.yscale('log')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.savefig('Fig1_Training_Curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: Fig1_Training_Curves.png")


In [63]:
# =============================================================================
# UPDATED PLOTTING FUNCTIONS (4-WAY COMPARISON)
# =============================================================================

def plot_fig2_single_condition_comparison(mlp_errors, pinn_errors, pinn_hard_errors, pf_errors):
    """Figure 2: Performance comparison including Hard PINN"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    data = [pf_errors, mlp_errors, pinn_errors, pinn_hard_errors]
    labels = ['Particle\nFilter', 'MLP\n(5k Data)', 'PINN\n(Soft)', 'PINN\n(Hard)']
    colors = ['#95B8D1', '#FF6B6B', '#4ECDC4', '#45B7D1'] # Hard PINN is darker Cyan/Blue
    
    # Box plot
    bp = ax1.boxplot(data, labels=labels, patch_artist=True, widths=0.6)
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax1.set_ylabel('Localization Error (μm)', fontsize=13, fontweight='bold')
    ax1.set_title('(a) Error Distribution (Same Condition)', fontsize=14, fontweight='bold')
    ax1.grid(True, axis='y', alpha=0.3, linestyle='--')
    
    # Bar chart
    means = [np.mean(d) for d in data]
    stds = [np.std(d) for d in data]
    
    bars = ax2.bar(labels, means, yerr=stds, capsize=8, width=0.6, 
                   color=colors, alpha=0.7, edgecolor='black')
    
    ax2.set_ylabel('Mean Error (μm)', fontsize=13, fontweight='bold')
    ax2.set_title('(b) Mean Performance', fontsize=14, fontweight='bold')
    ax2.grid(True, axis='y', alpha=0.3, linestyle='--')
    
    for bar, mean, std in zip(bars, means, stds):
        ax2.text(bar.get_x() + bar.get_width()/2., mean + std + 0.1,
                f'{mean:.1f}±{std:.1f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('Fig2_Single_Condition_Comparison.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig2_Single_Condition_Comparison.png")

def plot_fig5a_generalization_bars(mlp_base, pinn_base, pinn_hard_base, gen_results):
    conditions = ['Same (control)'] + list(gen_results.keys())
    means = {
        'MLP': [np.mean(mlp_base)],
        'PINN (Soft)': [np.mean(pinn_base)],
        'PINN (Hard)': [np.mean(pinn_hard_base)]
    }
    stds = {
        'MLP': [np.std(mlp_base)],
        'PINN (Soft)': [np.std(pinn_base)],
        'PINN (Hard)': [np.std(pinn_hard_base)]
    }
    
    for c in gen_results:
        means['MLP'].append(gen_results[c]['mlp_mean'])
        stds['MLP'].append(gen_results[c]['mlp_std'])
        means['PINN (Soft)'].append(gen_results[c]['pinn_mean'])
        stds['PINN (Soft)'].append(gen_results[c]['pinn_std'])
        means['PINN (Hard)'].append(gen_results[c]['pinn_hard_mean'])
        stds['PINN (Hard)'].append(gen_results[c]['pinn_hard_std'])

    plt.figure(figsize=(12, 6))
    x = np.arange(len(conditions))
    width = 0.25
    
    plt.bar(x - width, means['MLP'], width, yerr=stds['MLP'], label='MLP (Big Data)', 
            color='#FF6B6B', alpha=0.8, capsize=5, edgecolor='black')
    plt.bar(x, means['PINN (Soft)'], width, yerr=stds['PINN (Soft)'], label='PINN (Soft)', 
            color='#4ECDC4', alpha=0.8, capsize=5, edgecolor='black')
    plt.bar(x + width, means['PINN (Hard)'], width, yerr=stds['PINN (Hard)'], label='PINN (Hard)', 
            color='#45B7D1', alpha=0.8, capsize=5, edgecolor='black')

    plt.xlabel('Test Condition', fontsize=14, fontweight='bold')
    plt.ylabel('Error (μm)', fontsize=14, fontweight='bold')
    plt.title('Generalization: MLP vs PINN', fontsize=16, fontweight='bold')
    plt.xticks(x, conditions, rotation=15)
    plt.legend()
    plt.grid(True, axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('Fig5a_Generalization_Bars.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig5a_Generalization_Bars.png")

def plot_fig5b_generalization_degradation(mlp_base, pinn_base, pinn_hard_base, gen_results):
    conditions = ['Same (control)'] + list(gen_results.keys())
    m_mlp = [np.mean(mlp_base)] + [gen_results[c]['mlp_mean'] for c in gen_results]
    m_hard = [np.mean(pinn_hard_base)] + [gen_results[c]['pinn_hard_mean'] for c in gen_results]
    
    norm_mlp = np.array(m_mlp) / (m_mlp[0] + 1e-10)
    norm_hard = np.array(m_hard) / (m_hard[0] + 1e-10)
    
    plt.figure(figsize=(10, 6))
    plt.plot(conditions, norm_mlp, 'o-', label='MLP', color='#FF6B6B', lw=3, markersize=10)
    plt.plot(conditions, norm_hard, '^-', label='PINN (Hard)', color='#45B7D1', lw=3, markersize=10)
    
    plt.axhline(1.0, color='k', linestyle='--', alpha=0.3)
    plt.ylabel('Normalized Error', fontweight='bold')
    plt.title('Robustness Analysis', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('Fig5b_Generalization_Degradation.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig5b_Generalization_Degradation.png")


def plot_fig7a_error_vs_distance(true_sources, mlp_e, pinn_e, hard_e, pf_e):
    """Figure 7a with all 4 methods"""
    center = np.array([0.5, 0.5])
    dists = np.linalg.norm(true_sources - center, axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(dists, pf_e, alpha=0.6, label='Particle Filter', color='gray')
    plt.scatter(dists, pinn_e, alpha=0.6, label='PINN (Soft)', color='#4ECDC4')
    plt.scatter(dists, mlp_e, alpha=0.6, label='MLP (5k)', color='#FF6B6B')
    plt.scatter(dists, hard_e, alpha=0.8, marker='^', s=80, label='PINN (Hard)', color='#45B7D1', edgecolors='k')
    
    plt.xlabel('Distance from Center')
    plt.ylabel('Error (μm)')
    plt.title('Spatial Bias Analysis', fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('Fig7a_Error_vs_Distance.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig7a_Error_vs_Distance.png")



In [64]:

def plot_fig3_spatial_error_maps(true_sources, mlp_preds, pinn_preds, 
                                 mlp_errors, pinn_errors):
    """Figure 3: Spatial error visualization"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    vmax = max(mlp_errors.max(), pinn_errors.max())
    
    # MLP
    scatter1 = ax1.scatter(true_sources[:, 0], true_sources[:, 1], 
                          c=mlp_errors, cmap='Reds', s=150, 
                          edgecolor='black', linewidth=1.5, vmin=0, vmax=vmax)
    for i in range(len(true_sources)):
        ax1.arrow(true_sources[i, 0], true_sources[i, 1],
                 mlp_preds[i, 0] - true_sources[i, 0], 
                 mlp_preds[i, 1] - true_sources[i, 1],
                 color='blue', alpha=0.4, width=0.008, head_width=0.03)
    ax1.set_xlabel('X Position', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Y Position', fontsize=13, fontweight='bold')
    ax1.set_title('(a) Pure ML (MLP) Predictions', fontsize=14, fontweight='bold')
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.grid(True, alpha=0.3, linestyle='--')
    cbar1 = plt.colorbar(scatter1, ax=ax1, pad=0.02)
    cbar1.set_label('Error (μm)', fontsize=12, fontweight='bold')
    
    # PINN
    scatter2 = ax2.scatter(true_sources[:, 0], true_sources[:, 1], 
                          c=pinn_errors, cmap='Greens', s=150, 
                          edgecolor='black', linewidth=1.5, vmin=0, vmax=vmax)
    for i in range(len(true_sources)):
        ax2.arrow(true_sources[i, 0], true_sources[i, 1],
                 pinn_preds[i, 0] - true_sources[i, 0], 
                 pinn_preds[i, 1] - true_sources[i, 1],
                 color='blue', alpha=0.4, width=0.008, head_width=0.03)
    ax2.set_xlabel('X Position', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Y Position', fontsize=13, fontweight='bold')
    ax2.set_title('(b) Inverse PINN Predictions', fontsize=14, fontweight='bold')
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.grid(True, alpha=0.3, linestyle='--')
    cbar2 = plt.colorbar(scatter2, ax=ax2, pad=0.02)
    cbar2.set_label('Error (μm)', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('Fig3_Spatial_Error_Maps.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: Fig3_Spatial_Error_Maps.png")


In [65]:

def plot_fig4_true_vs_predicted(true_sources, mlp_preds, pinn_preds):
    """Figure 4: True vs predicted scatter plots"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # MLP
    ax1.scatter(true_sources[:, 0], mlp_preds[:, 0], 
               alpha=0.7, s=80, label='X coordinate', color='#E63946')
    ax1.scatter(true_sources[:, 1], mlp_preds[:, 1], 
               alpha=0.7, s=80, label='Y coordinate', color='#457B9D')
    ax1.plot([0, 1], [0, 1], 'k--', alpha=0.4, linewidth=2, label='Perfect prediction')
    ax1.set_xlabel('True Position', fontsize=13, fontweight='bold')
    ax1.set_ylabel('MLP Predicted Position', fontsize=13, fontweight='bold')
    ax1.set_title('(a) Pure ML Performance', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.set_aspect('equal')
    
    # PINN
    ax2.scatter(true_sources[:, 0], pinn_preds[:, 0], 
               alpha=0.7, s=80, label='X coordinate', color='#E63946')
    ax2.scatter(true_sources[:, 1], pinn_preds[:, 1], 
               alpha=0.7, s=80, label='Y coordinate', color='#457B9D')
    ax2.plot([0, 1], [0, 1], 'k--', alpha=0.4, linewidth=2, label='Perfect prediction')
    ax2.set_xlabel('True Position', fontsize=13, fontweight='bold')
    ax2.set_ylabel('PINN Predicted Position', fontsize=13, fontweight='bold')
    ax2.set_title('(b) Inverse PINN Performance', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.set_aspect('equal')
    
    plt.tight_layout()
    plt.savefig('Fig4_True_vs_Predicted.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: Fig4_True_vs_Predicted.png")


In [66]:

def plot_fig6_concentration_field_example(solver, sensors):
    """Figure 6: Example concentration field with sensor locations"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    examples = [(0.3, 0.3), (0.5, 0.5), (0.7, 0.6)]
    titles = ['(a) Source: Lower Left', '(b) Source: Center', '(c) Source: Upper Right']
    
    for ax, (xs, ys), title in zip(axes, examples, titles):
        C = solver.solve(xs, ys)
        
        im = ax.imshow(C, origin='lower', cmap='hot', extent=[0, 1, 0, 1], 
                      interpolation='bilinear')
        ax.plot(xs, ys, 'g*', markersize=25, markeredgecolor='white', 
               markeredgewidth=2.5, label='Source', zorder=10)
        
        for loc in sensors.locations:
            ax.plot(loc[0], loc[1], 'wo', markersize=10, markeredgecolor='black', 
                   markeredgewidth=2, zorder=9)
        
        ax.set_xlabel('X Position', fontsize=12, fontweight='bold')
        ax.set_ylabel('Y Position', fontsize=12, fontweight='bold')
        ax.set_title(title, fontsize=13, fontweight='bold')
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('Concentration', fontsize=11, fontweight='bold')
    
    # Add legend to first subplot
    axes[0].legend(loc='upper left', fontsize=11, framealpha=0.9)
    
    plt.tight_layout()
    plt.savefig('Fig6_Concentration_Fields.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Saved: Fig6_Concentration_Fields.png")


In [67]:
def plot_fig5c_generalization_boxplots(mlp_base, pinn_base, pinn_hard_base, gen_results):
    """Figure 5c: Error distributions across conditions (4-Way)"""
    # Prepare data structure
    data_to_plot = [mlp_base, pinn_base, pinn_hard_base]
    labels = ['Same\n(MLP)', 'Same\n(Soft)', 'Same\n(Hard)']
    
    # Iterate through generalization conditions
    for cond in gen_results:
        short_name = cond.split('(')[0].strip()
        data_to_plot.extend([
            gen_results[cond]['mlp_errors'], 
            gen_results[cond]['pinn_errors'],
            gen_results[cond]['pinn_hard_errors']
        ])
        labels.extend([
            f'{short_name}\n(MLP)', 
            f'{short_name}\n(Soft)',
            f'{short_name}\n(Hard)'
        ])

    plt.figure(figsize=(14, 6))
    bp = plt.boxplot(data_to_plot, labels=labels, patch_artist=True, widths=0.7)
    
    # Color coding: Red (MLP), Teal (Soft), Cyan (Hard)
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1'] * (1 + len(gen_results))
    
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
        patch.set_edgecolor('black')
    
    plt.ylabel('Localization Error (μm)', fontsize=14, fontweight='bold')
    plt.title('Error Distributions: Big Data MLP vs Physics', fontsize=16, fontweight='bold', pad=15)
    plt.grid(True, axis='y', alpha=0.3, linestyle='--')
    plt.xticks(fontsize=9, rotation=0)
    plt.tight_layout()
    plt.savefig('Fig5c_Generalization_Boxplots.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig5c_Generalization_Boxplots.png")

def plot_fig5d_generalization_summary(mlp_base, pinn_base, pinn_hard_base, gen_results):
    """Figure 5d: Summary table (4-Way)"""
    conditions = ['Same (control)'] + list(gen_results.keys())
    
    # Collect Means
    mlp_means = [np.mean(mlp_base)] + [gen_results[c]['mlp_mean'] for c in gen_results]
    hard_means = [np.mean(pinn_hard_base)] + [gen_results[c]['pinn_hard_mean'] for c in gen_results]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('off')
    
    # Headers
    table_data = [['Condition', 'MLP (5k) Error', 'Hard PINN Error', 'PINN Advantage']]
    
    for i, cond in enumerate(conditions):
        mlp_err = f'{mlp_means[i]:.2f} μm'
        hard_err = f'{hard_means[i]:.2f} μm'
        
        # Calculate advantage (avoid divide by zero)
        if hard_means[i] > 0.001:
            adv_val = mlp_means[i] / hard_means[i]
            advantage = f'{adv_val:.1f}×'
        else:
            advantage = "∞"
            
        table_data.append([cond, mlp_err, hard_err, advantage])
    
    # Draw Table
    table = ax.table(cellText=table_data, cellLoc='center', loc='center',
                     colWidths=[0.3, 0.2, 0.2, 0.15])
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2.5)
    
    # Styling
    for j in range(4):
        table[(0, j)].set_facecolor('#2C3E50')
        table[(0, j)].set_text_props(color='white', weight='bold')
    
    for i in range(1, len(table_data)):
        # Highlight the "Advantage" column
        table[(i, 3)].set_facecolor('#FFF9C4')
        table[(i, 3)].set_text_props(weight='bold')

    plt.title('Performance Summary: MLP vs Hard PINN', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('Fig5d_Generalization_Summary.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig5d_Generalization_Summary.png")

def plot_fig7b_error_cdf(mlp_err, pinn_err, hard_err, pf_err):
    """Figure 7b: CDF with all 4 methods"""
    plt.figure(figsize=(10, 6))
    
    methods = [
        ('Particle Filter', pf_err, 'gray'),
        ('MLP (5k Data)', mlp_err, '#FF6B6B'),
        ('PINN (Soft)', pinn_err, '#4ECDC4'),
        ('PINN (Hard)', hard_err, '#45B7D1')
    ]
    
    for label, data, color in methods:
        sorted_data = np.sort(data)
        yvals = np.arange(len(sorted_data)) / float(len(sorted_data) - 1)
        plt.plot(sorted_data, yvals, label=label, linewidth=3, color=color, alpha=0.8)
    
    plt.xlabel('Error (μm)', fontsize=14, fontweight='bold')
    plt.ylabel('Cumulative Probability', fontsize=14, fontweight='bold')
    plt.title('Error CDF: Which method is most reliable?', fontsize=16, fontweight='bold')
    plt.legend(loc='lower right', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.savefig('Fig7b_Error_CDF.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig7b_Error_CDF.png")

def plot_fig7c_error_histogram(mlp_err, pinn_err, hard_err, pf_err):
    """Figure 7c: Histogram with all 4 methods"""
    plt.figure(figsize=(10, 6))
    
    # Determine bins based on max error
    max_val = max(np.max(mlp_err), np.max(hard_err))
    bins = np.linspace(0, max_val + 0.5, 15)
    
    plt.hist(pf_err, bins, alpha=0.3, label='PF', color='gray', edgecolor='black')
    plt.hist(mlp_err, bins, alpha=0.5, label='MLP', color='#FF6B6B', edgecolor='black')
    # Use 'step' type for PINNs to avoid clutter
    plt.hist(pinn_err, bins, histtype='step', linewidth=2, label='Soft PINN', color='#4ECDC4')
    plt.hist(hard_err, bins, histtype='step', linewidth=3, label='Hard PINN', color='#45B7D1', linestyle='--')
    
    plt.xlabel('Error (μm)', fontsize=14, fontweight='bold')
    plt.ylabel('Count', fontsize=14, fontweight='bold')
    plt.title('Error Histogram', fontsize=16, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('Fig7c_Error_Histogram.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig7c_Error_Histogram.png")

def plot_fig7d_statistical_summary(mlp_err, pinn_err, hard_err, pf_err):
    """Figure 7d: Detailed stats table (4-Way)"""
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('off')
    
    cols = ['Metric', 'PF', 'MLP', 'Soft PINN', 'Hard PINN']
    metrics = ['Mean', 'Median', 'Std Dev', 'Min', 'Max']
    
    table_data = [cols]
    
    datasets = [pf_err, mlp_err, pinn_err, hard_err]
    
    for m in metrics:
        row = [m]
        for data in datasets:
            if m == 'Mean': val = np.mean(data)
            elif m == 'Median': val = np.median(data)
            elif m == 'Std Dev': val = np.std(data)
            elif m == 'Min': val = np.min(data)
            elif m == 'Max': val = np.max(data)
            row.append(f'{val:.3f}')
        table_data.append(row)
        
    table = ax.table(cellText=table_data, cellLoc='center', loc='center',
                     colWidths=[0.2, 0.15, 0.15, 0.2, 0.2])
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1, 2)
    
    # Headers
    for j in range(5):
        table[(0, j)].set_facecolor('#2980B9')
        table[(0, j)].set_text_props(color='white', weight='bold')
        
    # Best Performer Highlight (Columns 2,3,4 correspond to MLP, Soft, Hard)
    # Row indices 1-5. Let's find best mean (Row 1)
    means = [float(table_data[1][i]) for i in range(1,5)]
    best_idx = means.index(min(means)) + 1 # +1 offset for label col
    table[(1, best_idx)].set_facecolor('#C8E6C9') # Light Green for best mean
        
    plt.title('Statistical Breakdown', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('Fig7d_Statistical_Summary.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig7d_Statistical_Summary.png")

In [68]:
def plot_fig8_uncertainty_analysis(pinn_hard_uncertainties):
    """Figure 8: Hard PINN uncertainty quantification"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Scatter: X vs Y uncertainty
    x_unc = [u[0] for u in pinn_hard_uncertainties]
    y_unc = [u[1] for u in pinn_hard_uncertainties]
    
    ax1.scatter(x_unc, y_unc, alpha=0.6, s=100, c='#45B7D1', edgecolor='black')
    ax1.set_xlabel('X Position Uncertainty', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Y Position Uncertainty', fontsize=13, fontweight='bold')
    ax1.set_title('(a) Uncertainty Distribution', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Histogram of total uncertainty
    total_unc = [np.sqrt(u[0]**2 + u[1]**2) for u in pinn_hard_uncertainties]
    ax2.hist(total_unc, bins=10, alpha=0.7, color='#45B7D1', edgecolor='black')
    ax2.set_xlabel('Total Uncertainty (μm)', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Count', fontsize=13, fontweight='bold')
    ax2.set_title(f'(b) Mean Uncertainty: {np.mean(total_unc):.3f} μm', 
                  fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('Fig8_Uncertainty_Analysis.png', dpi=300)
    plt.close()
    print("✓ Saved: Fig8_Uncertainty_Analysis.png")

### MAIN

In [69]:

# =============================================================================
# MAIN EXPERIMENT
# =============================================================================

def run_complete_analysis():
    """Run complete analysis with separate figure generation"""
    
    print("="*70)
    print(" COMPLETE ODOR SOURCE LOCALIZATION ANALYSIS ".center(70))
    print("="*70)
    
    np.random.seed(42)
    
    # =========================================================================
    # PHASE 1: Single Condition Analysis
    # =========================================================================
    
    print("\n" + "="*70)
    print(" PHASE 1: SINGLE CONDITION ANALYSIS ".center(70))
    print("="*70)
    
    train_wind = 0.001
    train_angle = 45
    
    print(f"\nTraining condition: {train_wind*1000:.1f} mm/s @ {train_angle}°")
    
    solver = PDESolver(train_wind, train_angle)
    sensors = SensorNetwork(n_sensors=9)
    
    # Learnability analysis
    learnability = analyze_learnability(solver, sensors)
    
    # Generate dataset
    print("\n" + "-"*70)

    # ... inside run_complete_analysis ...
    # TARGET: Generate enough data to learn the inverse physics mapping
    N_SAMPLES = 5000  # Increased from 150 to 5000
    print(f"Generating massive dataset ({N_SAMPLES} samples)...")

    X_all, Y_all = [], []
    for i in range(N_SAMPLES):
    # Random source location
        xs, ys = np.random.uniform(0.2, 0.8, 2)
    
    # Randomize wind slightly during training to help generalization (Optional but recommended)
    # If you want to strictly match the "Control" condition, keep wind fixed.
    # But to beat the "Generalization" test, you should train on varied winds.
    # For now, let's stick to the paper's rules: Fixed Wind, just MORE data.
        C = solver.solve(xs, ys)
        meas = sensors.measure(C, noise=0.05)
    
        X_all.append(meas)
        Y_all.append([xs, ys])
    
        if (i+1) % 500 == 0:
            print(f"  {i+1}/{N_SAMPLES}")

    X_all = np.array(X_all)
    Y_all = np.array(Y_all)

    # Train/val split (80/20)
    split = int(0.8 * N_SAMPLES)
    X_train, X_val = X_all[:split], X_all[split:]
    Y_train, Y_val = Y_all[:split], Y_all[split:]

    print(f"\nTrain: {len(X_train)}, Val: {len(X_val)}")


    #print("Generating dataset (150 samples)...")
    #X_all, Y_all = [], []
    #for i in range(150):
    #    xs, ys = np.random.uniform(0.2, 0.8, 2)
    #    C = solver.solve(xs, ys)
    #    meas = sensors.measure(C, noise=0.05)
    #    X_all.append(meas)
    #    Y_all.append([xs, ys])
    #    if (i+1) % 50 == 0:
    #        print(f"  {i+1}/150")
    
    #X_all = np.array(X_all)
    #Y_all = np.array(Y_all)
    
    # Train/val split
    #split = 120
    #X_train, X_val = X_all[:split], X_all[split:]
    #Y_train, Y_val = Y_all[:split], Y_all[split:]
    
    #print(f"\nTrain: {len(X_train)}, Val: {len(X_val)}")
    
    # Train MLP
    print("\nTraining MLP...")
    mlp = MLP(sensors.n_sensors)
    mlp.train(X_train, Y_train, X_val, Y_val, epochs=150)
    
    print("\n" + "="*70)
    print(" VALIDATION LOSS DIAGNOSTIC ".center(70))
    print("="*70)

    # 1. Check if validation loss values are actually changing
    print("\n1. Last 20 validation loss values:")
    print(mlp.val_losses[-20:])
    print(f"   Min: {min(mlp.val_losses):.8f}")
    print(f"   Max: {max(mlp.val_losses):.8f}")
    print(f"   Range: {max(mlp.val_losses) - min(mlp.val_losses):.8f}")

    # 2. Check validation predictions
    print("\n2. Validation predictions analysis:")
    X_val_subset = X_val[:10]  # First 10 samples
    val_predictions = mlp.predict(X_val_subset)
    print(f"   Prediction means: {np.mean(val_predictions, axis=0)}")
    print(f"   Prediction stds: {np.std(val_predictions, axis=0)}")
    print(f"   First 5 predictions:")
    for i in range(5):
        print(f"     Sample {i}: {val_predictions[i]}")

    # 3. Check if all predictions are identical
    unique_x = len(np.unique(np.round(val_predictions[:, 0], 4)))
    unique_y = len(np.unique(np.round(val_predictions[:, 1], 4)))
    print(f"\n3. Prediction diversity (rounded to 4 decimals):")
    print(f"   Unique X predictions: {unique_x}/10")
    print(f"   Unique Y predictions: {unique_y}/10")

    # 4. Check output layer weights
    print("\n4. Output layer analysis:")
    print(f"   W3 mean: {np.mean(mlp.W3):.6f}, std: {np.std(mlp.W3):.6f}")
    print(f"   b3 values: {mlp.b3}")
    print(f"   Output logits for center input would be: {mlp.b3}")
    print(f"   Output sigmoid of logits: {mlp.sigmoid(mlp.b3)}")

    # 5. Plot validation loss with higher resolution
    print("\n5. Generating high-resolution validation loss plot...")
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(mlp.val_losses, linewidth=1, alpha=0.8, color='purple')
    plt.ylabel('Validation Loss', fontweight='bold')
    plt.title('Validation Loss (Linear Scale)', fontweight='bold')
    plt.grid(True, alpha=0.3)

    plt.subplot(2, 1, 2)
    plt.plot(mlp.val_losses, linewidth=1, alpha=0.8, color='purple')
    plt.ylabel('Validation Loss (log)', fontweight='bold')
    plt.xlabel('Epoch', fontweight='bold')
    plt.title('Validation Loss (Log Scale)', fontweight='bold')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('Diagnostic_Validation_Loss.png', dpi=300)
    plt.close()
    print("   Saved: Diagnostic_Validation_Loss.png")

    print("="*70)
    # Test on held-out data
    print("\n" + "-"*70)
    print("Testing on 20 held-out samples...")
    
    np.random.seed(999)
    test_sources = [(np.random.uniform(0.2, 0.8), np.random.uniform(0.2, 0.8)) 
                   for _ in range(20)]
    
    pinn = PINN(solver, sensors)
    pinn_hard = PINN_hard(solver, sensors)
    pf = ParticleFilter(solver, sensors)
    
    mlp_errors_single = []
    pinn_errors_single = []
    pinn_hard_errors_single = []
    pf_errors_single = []
    mlp_preds_single = []
    pinn_preds_single = []
    pinn_hard_preds_single = []
    pf_preds_single = []
    true_sources_single = []
    all_std = []
    
    for i, (xs_true, ys_true) in enumerate(test_sources):
        C_true = solver.solve(xs_true, ys_true)
        meas = sensors.measure(C_true, noise=0.05)
        
        mlp_pred = mlp.predict(meas.reshape(1, -1))[0]

        # In the test loop, before pinn_pred = pinn.predict(meas)
        print(f"\nDEBUG Sample {i}:")
        print(f"  Measurements: {meas}")
        print(f"  True source: ({xs_true:.3f}, {ys_true:.3f})")

        # Test the PDE residual calculation
        C_test = solver.solve(0.5, 0.5)
        residual_test = pinn.compute_pde_residual(C_test, solver.dx)
        print(f"  Test residual: {residual_test}")

        
        pinn_pred = pinn.predict(meas)
        #pinn_hard_pred = pinn_hard.predict(meas)

        # Ensemble prediction with RANDOM initializations
        pinn_hard_preds_ensemble = []
        for _ in range(5):
            # Temporarily replace predict with random start
            best, best_loss = None, np.inf
            # Random starting points
            starts = [np.random.uniform(0.2, 0.8, 2) for _ in range(3)]
            
            for x0 in starts:
                res = minimize(lambda p: pinn_hard.loss(p, meas), x0, 
                            method='Nelder-Mead', options={'maxiter': 40})
                if res.fun < best_loss: 
                    best_loss = res.fun
                    best = res
            
            pred = best.x if best else np.array([0.5, 0.5])
            pinn_hard_preds_ensemble.append(pred)
        """
        pinn_hard_preds_ensemble = []
        for _ in range(5):  # 5 runs with different initializations
            pred = pinn_hard.predict(meas)
            pinn_hard_preds_ensemble.append(pred)
        """
        pinn_hard_pred = np.mean(pinn_hard_preds_ensemble, axis=0)
        pinn_hard_std = np.std(pinn_hard_preds_ensemble, axis=0)
        pf_pred = pf.predict(meas)
        
        all_std.append(pinn_hard_std)  # Add this line
        
        mlp_error = np.linalg.norm(mlp_pred - [xs_true, ys_true]) * 10
        pinn_error = np.linalg.norm(pinn_pred - [xs_true, ys_true]) * 10
        pinn_hard_error = np.linalg.norm(pinn_hard_pred - [xs_true, ys_true]) * 10
        pf_error = np.linalg.norm(pf_pred - [xs_true, ys_true]) * 10
        
        mlp_errors_single.append(mlp_error)
        pinn_errors_single.append(pinn_error)
        pinn_hard_errors_single.append(pinn_hard_error) 
        pf_errors_single.append(pf_error)
        mlp_preds_single.append(mlp_pred)
        pinn_preds_single.append(pinn_pred)
        pinn_hard_preds_single.append(pinn_hard_pred)
        pf_preds_single.append(pf_pred)
        true_sources_single.append([xs_true, ys_true])
        
        if (i+1) % 5 == 0:
            print(f"  {i+1}/20")
    
    # After all test samples (outside the loop)
    print(f"\nHard PINN Uncertainty Analysis:")
    print(f"  Mean X uncertainty: {np.mean([std[0] for std in all_std]):.4f}")
    print(f"  Mean Y uncertainty: {np.mean([std[1] for std in all_std]):.4f}")
    
    mlp_errors_single = np.array(mlp_errors_single)
    pinn_errors_single = np.array(pinn_errors_single)
    pinn_hard_errors_single = np.array(pinn_hard_errors_single)
    
    pf_errors_single = np.array(pf_errors_single)
    mlp_preds_single = np.array(mlp_preds_single)
    pinn_preds_single = np.array(pinn_preds_single)
    pinn_hard_preds_single = np.array(pinn_hard_preds_single)
    pf_preds_single = np.array(pf_preds_single)
    true_sources_single = np.array(true_sources_single)
    
    print(f"\nResults on SAME condition:")
    print(f"  PF:   {np.mean(pf_errors_single):.1f} ± {np.std(pf_errors_single):.1f} μm")
    print(f"  MLP:  {np.mean(mlp_errors_single):.1f} ± {np.std(mlp_errors_single):.1f} μm")
    print(f"  PINN: {np.mean(pinn_errors_single):.1f} ± {np.std(pinn_errors_single):.1f} μm")
    print(f"  PINN: {np.mean(pinn_hard_errors_single):.1f} ± {np.std(pinn_hard_errors_single):.1f} μm")
    # =========================================================================
    # PHASE 2: GENERALIZATION TEST
    # =========================================================================
    
    print("\n" + "="*70)
    print(" PHASE 2: GENERALIZATION TEST ".center(70))
    print("="*70)
    
    test_conditions = [
        ("Higher wind (50%)", 0.0015, 45),
        ("Different angle", 0.001, 30),
        ("Both changed", 0.0012, 60),
    ]
    
    generalization_results = {}
    
    for cond_name, test_wind, test_angle in test_conditions:
        print(f"\n{cond_name}: {test_wind*1000:.1f} mm/s @ {test_angle}°")
        
        test_solver = PDESolver(test_wind, test_angle)
        test_pinn = PINN(test_solver, sensors)
        test_pinn_hard = PINN_hard(test_solver, sensors)

        mlp_errors_gen = []
        pinn_errors_gen = []
        pinn_hard_errors_gen = []

        for i, (xs_true, ys_true) in enumerate(test_sources):
            C_true = test_solver.solve(xs_true, ys_true)
            meas = sensors.measure(C_true, noise=0.05)
            
            mlp_pred = mlp.predict(meas.reshape(1, -1))[0]
            mlp_error = np.linalg.norm(mlp_pred - [xs_true, ys_true]) * 10
            mlp_errors_gen.append(mlp_error)
            
            pinn_pred = test_pinn.predict(meas)
            pinn_error = np.linalg.norm(pinn_pred - [xs_true, ys_true]) * 10
            pinn_errors_gen.append(pinn_error)

            pinn_hard_pred = test_pinn_hard.predict(meas)
            pinn_hard_error = np.linalg.norm(pinn_hard_pred - [xs_true, ys_true]) * 10
            pinn_hard_errors_gen.append(pinn_hard_error)
            
            if (i+1) % 5 == 0:
                print(f"  {i+1}/20")

            if (i+1) % 5 == 0:
                print(f"  {i+1}/20 | Hard PINN unc: ±{np.linalg.norm(pinn_hard_std)*10:.3f} μm")
                
        mlp_errors_gen = np.array(mlp_errors_gen)
        pinn_errors_gen = np.array(pinn_errors_gen)
        pinn_hard_errors_gen = np.array(pinn_hard_errors_gen)


        generalization_results[cond_name] = {
            'mlp_mean': np.mean(mlp_errors_gen),
            'mlp_std': np.std(mlp_errors_gen),
            
            'pinn_mean': np.mean(pinn_errors_gen),
            'pinn_std': np.std(pinn_errors_gen),
            
            # --- CORRECTED SECTION ---
            'pinn_hard_mean': np.mean(pinn_hard_errors_gen),
            'pinn_hard_std': np.std(pinn_hard_errors_gen),   # <--- This line was missing!
            'pinn_hard_errors': pinn_hard_errors_gen,        # <--- This is needed for boxplots
            # -------------------------
            
            'mlp_errors': mlp_errors_gen,
            'pinn_errors': pinn_errors_gen
        }

        
        
        improvement_pınn = np.mean(mlp_errors_gen) / np.mean(pinn_errors_gen)
        improvement_pınn_hard = np.mean(mlp_errors_gen) / np.mean(pinn_hard_errors_gen)
        
        print(f"Results:")
        print(f"  MLP:  {np.mean(mlp_errors_gen):.1f} ± {np.std(mlp_errors_gen):.1f} μm")
        print(f"  PINN: {np.mean(pinn_errors_gen):.1f} ± {np.std(pinn_errors_gen):.1f} μm")
        print(f"  PINN is {improvement_pınn:.1f}× better!")

        print(f"Results:")
        print(f"  MLP:  {np.mean(mlp_errors_gen):.1f} ± {np.std(mlp_errors_gen):.1f} μm")
        print(f"  PINN Hard: {np.mean(pinn_hard_errors_gen):.1f} ± {np.std(pinn_hard_errors_gen):.1f} μm")
        print(f"  PINN Hard is {improvement_pınn_hard:.1f}× better!")
    
    # =========================================================================
    # PHASE 3: GENERATE ALL FIGURES
    # =========================================================================
    
    print("\n" + "="*70)
    print(" GENERATING PUBLICATION FIGURES ".center(70))
    print("="*70 + "\n")
    

    plot_fig1_training_curves(mlp)
    
    # Updated to take 4 arguments
    plot_fig2_single_condition_comparison(
        mlp_errors_single, 
        pinn_errors_single, 
        pinn_hard_errors_single, 
        pf_errors_single
    )
    
    # Updated to take 5 arguments (visualization of Hard PINN predictions)
    plot_fig3_spatial_error_maps(
        true_sources_single, 
        mlp_preds_single, 
        pinn_hard_preds_single,  # <--- Pass Hard PINN preds here
        mlp_errors_single, 
        pinn_hard_errors_single  # <--- Pass Hard PINN errors here
    )
    
    # Updated to take Hard PINN preds
    plot_fig4_true_vs_predicted(
        true_sources_single, 
        mlp_preds_single, 
        pinn_hard_preds_single   # <--- Compare MLP vs Hard PINN
    )
    
    # Updated for 4-way comparison
    plot_fig5a_generalization_bars(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, 
        generalization_results
    )
    
    plot_fig5b_generalization_degradation(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, 
        generalization_results
    )
    
    # NEW UPDATES FOR FIG 5c/5d
    plot_fig5c_generalization_boxplots(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, 
        generalization_results
    )
    
    plot_fig5d_generalization_summary(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, 
        generalization_results
    )
    
    plot_fig6_concentration_field_example(solver, sensors)
    
    # Updated for 4-way
    plot_fig7a_error_vs_distance(
        true_sources_single, mlp_errors_single, 
        pinn_errors_single, pinn_hard_errors_single, pf_errors_single
    )
    
    plot_fig7b_error_cdf(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, pf_errors_single
    )
    
    plot_fig7c_error_histogram(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, pf_errors_single
    )
    
    plot_fig7d_statistical_summary(
        mlp_errors_single, pinn_errors_single, pinn_hard_errors_single, pf_errors_single
    )
    
    # NEW: Uncertainty analysis
    plot_fig8_uncertainty_analysis(all_std)
    # =========================================================================
    # FINAL SUMMARY
    # =========================================================================
    
    print("\n" + "="*70)
    print(" ANALYSIS COMPLETE ".center(70))
    print("="*70)
    
    print("\n📊 Generated Figures:")
    print("  • Fig1_Training_Curves.png - MLP convergence analysis")
    print("  • Fig2_Single_Condition_Comparison.png - Performance comparison")
    print("  • Fig3_Spatial_Error_Maps.png - Spatial error visualization")
    print("  • Fig4_True_vs_Predicted.png - Prediction accuracy")
    print("  • Fig5a_Generalization_Bars.png - Performance across conditions")
    print("  • Fig5b_Generalization_Degradation.png - Degradation analysis")
    print("  • Fig5c_Generalization_Boxplots.png - Error distributions")
    print("  • Fig5d_Generalization_Summary.png - Summary table")
    print("  • Fig6_Concentration_Fields.png - Example concentration profiles")
    print("  • Fig7a_Error_vs_Distance.png - Error vs location")
    print("  • Fig7b_Error_CDF.png - Cumulative distribution")
    print("  • Fig7c_Error_Histogram.png - Error histogram")
    print("  • Fig7d_Statistical_Summary.png - Detailed statistics")
    print(f"\n  Total: 13 publication-ready figures")
    
    print("\n🎯 Key Findings:")
    print(f"  • Learnability correlation: {learnability['correlation']:.3f} (well-posed)")
    print(f"  • Same condition: PINN {np.mean(mlp_errors_single)/np.mean(pinn_errors_single):.1f}× better")
    print(f"  • Generalization: PINN consistently superior across all conditions")
    print(f"  • Physics-informed methods essential for robust localization")
    
    print("\n" + "="*70)

if __name__ == "__main__":
    run_complete_analysis()

              COMPLETE ODOR SOURCE LOCALIZATION ANALYSIS              

                  PHASE 1: SINGLE CONDITION ANALYSIS                  

Training condition: 1.0 mm/s @ 45°

                        LEARNABILITY ANALYSIS                         

1. Sensor Discrimination Test
  Min distance: 1.932505
  Max distance: 3.198503
  Mean distance: 2.648983
  ✓ Sources well-separated

2. Noise Robustness Test
  Noise 1%: SNR = 30.85
  Noise 5%: SNR = 6.05
  Noise 10%: SNR = 3.03

3. Inverse Problem Uniqueness
  Correlation: 0.776
  ✓ Problem well-posed

----------------------------------------------------------------------
Generating massive dataset (5000 samples)...
  500/5000
  1000/5000
  1500/5000
  2000/5000
  2500/5000
  3000/5000
  3500/5000
  4000/5000
  4500/5000
  5000/5000

Train: 4000, Val: 1000

Training MLP...
  Epoch 50: Val Loss 0.00608
  Epoch 100: Val Loss 0.00317
  Epoch 150: Val Loss 0.00275

                      VALIDATION LOSS DIAGNOSTIC                      

1. L